# 单准则决策
本文档提供了单准则决策的基本函数：  
如有需求，请根据Template提供新的函数  
请准备好：  
1.决策方案| 行标  
2.自然状态| 列标  
3.损益值 | 元素  
以上三者由矩阵形式给出  
4.决策情报(自然状态概率） | 函数大类  
5.决策准则 | 函数小类

In [1]:
from numpy import *

In [2]:
sample=matrix('0 0 0 0 0; -10 50 50 50 50; -20 40 100 100 100; -30 30 90 150 150; -40 20 80 140 200')

In [3]:
print(sample)

[[  0   0   0   0   0]
 [-10  50  50  50  50]
 [-20  40 100 100 100]
 [-30  30  90 150 150]
 [-40  20  80 140 200]]


In [4]:
def Template(mat,decide,choose):
    #高阶函数，抽象了决策流程
    res = []
    decide(mat,res)
    best = choose(res)

    decision = [];
    for i in range(len(res)):
        if res[i] == best:
            decision.append(i)
    return best,decision
    

# 不确定性决策
不确定型决策是指决策者对决策环境一无所知，决策者只能凭借自己的主观倾向进行决策。由于决策者对待风险的主观态度不同，采用的决策准则就不同，这样对同一问题进行决策得到的决策结果也有所不同  
不确定型的决策问题有以下五种决策准则：悲观主义决策准则、乐观主义决策准则、最小机会损失决策准则、等可能性决策准则、折衷主义决策准则。  
  
  
对于不确定型决策问题，如果没有告诉决策者使用哪种决策准则进行决策，通常会使用五种决策准则分别进行决策，然后选取出现次数最多的最优方案为最终最优方案  
但在实际的决策问题中，人们会尽可能事先获取有关各自然状态发生的概率，使不确定型决策问题转化为风险型决策问题  



In [5]:
#分析各种最坏的可能结果，从中选择最好的结果，以它对应的策略为最优策略
def MaxMin(mat):
#悲观主义决策准则
#输入一个决策矩阵，输出最优的最坏情况以及最优决策的集合
    def Decision(mat,res):
        for r in mat:
            res.append(r.min())
    return Template(mat,Decision,max)

In [6]:
print(MaxMin(sample))

(0, [0])


In [7]:
#分析各种最好的可能结果，从中选择最好的结果，以它对应的策略为最优策略
def MaxMax(mat):
#乐观主义决策准则
#输入一个决策矩阵，输出最优的最佳情况以及最优决策的集合
    def Decision(mat,res):
        for r in mat:
            res.append(r.max())
    return Template(mat,Decision,max)


In [8]:
print(MaxMax(sample))

(200, [4])


In [9]:
#分析各种决策的期望益损，从中选择最好的结果，以它对应的策略为最优策略
def Laplace(mat):
#等可能性决策准则
#输入一个决策矩阵，输出最优期望以及最优决策的集合
    def Decision(mat,res):
        for r in mat:
            res.append(r.mean())
    return Template(mat,Decision,max)

In [10]:
print(Laplace(sample))

(80.0, [4])


In [11]:
#分析各种决策的机会损失，从中选择最好的结果，以它对应的策略为最优策略
def Savage(mat):
#最小机会损失决策准则
#输入一个决策矩阵，输出最小机会损失以及最优决策的集合
    def Decision(mat,res):
        mat_s = zeros(mat.shape)
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                mat_s[i,j] = mat[:,j].max() - mat[i,j]
        for r in mat_s:
            res.append(r.max())
    return Template(mat,Decision,min)

In [12]:
print(Savage(sample))

(40.0, [4])


In [13]:
#分析各种决策的最好结果和最坏结果的线性组合，从中选择最好的结果，以它对应的策略为最优策略
def Mean(mat,alpha):
#折中主义决策准则
#输入一个决策矩阵，输出折中最佳结果以及最优决策的集合
    def Decision(mat,res):
        for r in mat:
            res.append(alpha*r.max()+(1-alpha) * r.min())
    return Template(mat,Decision,max)

In [14]:
print(Mean(sample,0.5))

(80.0, [4])


# 风险性决策
风险型决策是指决策的环境不是完全确定的，但决策者面临的自然状态是已知的，即决策者可以根据过去的经验或主观估计等，得到每种状态出现可能性的一个主观概率。此时决策的结果受概率估计值的影响，因而对方案的选择带有一定的风险性  
对于风险型的决策问题，其决策方法有最大似然值决策准则、最大期望收益决策准则、最小期望机会损失决策准则、贝叶斯决策准则等  


In [15]:
pro=[0.1,0.2,0.4,0.2,0.1]

In [16]:
#最大似然值决策准则是在一组自然状态中某个状态出现的概率比其他状态大得多，而它们相应的益损值差别比较小的情况下所采用的一种方法
#该决策准则就是识别具有最大先验概率的状态，对于这一状态，找出具有最大益损值的决策方案
def ML(mat,pro):
#最大似然值决策准则
#输入一个决策矩阵，输出最大先验概率的最大益损值以及最优决策的集合
    def Decision(mat,res):
        maxpro=max(pro)
        for j in range(len(pro)):
            if pro[j]==maxpro:
                likely=j
        for i in range(mat.shape[0]):
            res.append(mat[i,likely])
    return Template(mat,Decision,max)

In [17]:
print(ML(sample,pro))

(100, [2])


In [18]:
#最大期望收益决策准则和用最小期望损失决策准则进行决策得到的结果是相同的，
#这表明ＥＭＶ准则和EOL准则是等价的。
#这两个决策准则适用于一次决策后多次重复进行应用的情况，它是平均意义下的最大收益或最小损失
def EMV(mat,pro):
#最大期望收益决策准则
#输入一个决策矩阵，输出最大期望收益以及最优决策的集合
    def Decision(mat,res):
        for i in range(mat.shape[0]):
            sum = 0
            for j in range(mat.shape[1]):
                sum = sum + pro[j] * mat[i,j]
            res.append(sum)
    return Template(mat,Decision,max)

In [19]:
print(EMV(sample,pro))

(84.0, [3])


In [20]:
def EOL(mat,pro):
#最小期望机会损失决策准则
#输入一个决策矩阵，输出最小期望机会损失以及最优决策的集合
    def Decision(mat,res):
        mat_s = zeros(mat.shape)
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                mat_s[i,j] = mat[:,j].max() - mat[i,j]
        for i in range(mat_s.shape[0]):
            sum = 0
            for j in range(mat_s.shape[1]):
                sum = sum + pro[j] * mat_s[i,j]
            res.append(sum)
    return Template(mat,Decision,min)

In [21]:
print(EOL(sample,pro))

(16.0, [3])


In [22]:
#贝叶斯决策树（注意和机器学习的应用场景存在不同）
#有些决策问题，当进行决策后又产生一些新情况，并需要进行新的决策，接着又有一些新情况，又需要进行新的决策,称之为序列决策
#其中每一层都进行了条件概率，即贝叶斯公式的计算
#描述序列决策的有力工具就是决策树，决策树是由决策节点，状态节点和结果节点构成的树形结构。
#对于序列决策，一般选用最大期望收益值或最大期望效用值作为决策准则
#贝叶斯决策树本质上是嵌套的EMV
#自底向上，将下一层的EMV作为上一层的益损值即可。
#由于树的深度和结构不明，此处不给出具体实现，请自行组合函数
#示例
node= matrix('0 1; 1 0')
pro_n= [0.4,0.6]
EMV_node, method=EMV(node,pro_n)

node_root=matrix([[EMV_node,0],[1,EMV_node]])
pro_r= [0.3,0.7]
EMV_root,method_root=EMV(node_root,pro_r)
#对于 0.4 0.6
# 0     0   1
# 1     1   0
#其EMV为0.6 > 0.4，即选择0方案
#对于  0.3 0.7
# 0    0.6  0
# 1    1   0.6
#其EMV为0.3+0.42=0.72 > 0.18，即选择1方案受益最大为0.72
print(EMV_root)
#在顶层的决策中，应该选择1方案
print(method_root)
#顶层选择了1方案的情况下，若自然状态为node分支，应该选择0方案。（另一种自然状态同样存在着决策选择，此处益损为常值故不进行决策） 
print(method)

0.72
[1]
[0]


# 确定性决策
确定型决策指决策者对未来可能发生的情况有十分确定的比较，可以直接根据完全确定的情况选择最满意的行动方案。

In [24]:
#全情报
#如果能够预测自然状态发生，这时所得到的期望收益值称为全情报的期望收益值EPPL,亦全局的最高收益
#此时EPPL-EMV=EVPI称为全情报的价值
#要进行调研必然要花费一定的费用，获取情报的费用不能超过EVPI值
#注意:这一点如果纳入考虑我觉得是亮点!
def EPPL(mat,pro):
    sum=0
    for j in range(mat.shape[1]):
        sum = sum + pro[j] * mat[:,j].max()
    return sum
def EVPI(mat,pro):
    return EPPL(mat,pro)-EMV(mat,pro)[0]

In [25]:
print(EPPL(sample,pro))
print(EVPI(sample,pro))

100.0
16.0


# 多准则决策（Multi-criteria decision-making, MCDM）
　多准则决策根据决策方案是有限还是无限，而分为多属性决策(MADM)与多目标决策(MODM )两大类。  

#### 1、多属性决策  

　　多属性决策也称有限方案多目标决策，是指在考虑多个属性的情况下，选择最优备选方案或进行方案排序的决策问题，它是现代决策科学的一个重要组成部分。它的理论和方法在工程、技术、经济、管理和军事等诸多领域中都有广泛的应用。

#### 2、多目标决策

　　多目标决策是指需要同时考虑两个或两个以上目标的决策。如某企业要在几种产品中选择一种产品生产，就既要考虑获利大小，又要考虑现有设备能否生产以及原材料供应是否充足等因素来选择其中一种，只有使这些相互联系和相互制约的因素都能得到最佳的协调、配合和满足，才是最优的决策。

与常规评估方法相比，多准则决策评估的机理性特点是：  

　　1、可进行多个项目的评判、排队和选优。  

　　2、对一个项目进行研究时，每个影响因子都以主判这个项目的准则对待，并对因子的数值要进行一系列的信息加工和提取，给各因子的重要性赋以权重。  

　　3、将项目评估表看做是一个决策判断矩阵，然后采用十多种决策判别方法组织信息，组合成一个分析机理性强的动态分析系统，进而借助于现代化的计算机技术快速完成信息加工；还可组织决策会议，快速反应决策参与人的意见，利于形成一致性观点。
  
MCMD主要是通过专家问卷法获取权重，其中涉及权重计算的方法有AHP，ANP，灰关联，BWM等方法，用VIKOR, TOPSIS,灰关联等方法做绩效评估  
使用评价算法如AHP/ANP确定权重后，计算各属性的加权损益，即可转换为单准则决策。